In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
%ls

 edm_output.mid
'EDM Samples'/
 edm_weights-improvement-01-2.7717-bigger.hdf5
 edm_weights-improvement-01-3.0686-bigger.hdf5
 edm_weights-improvement-01-3.0734-bigger.hdf5
 edm_weights-improvement-01-3.0966-bigger.hdf5
 edm_weights-improvement-01-3.1132-bigger.hdf5
 edm_weights-improvement-01-3.1238-bigger.hdf5
 edm_weights-improvement-01-3.1257-bigger.hdf5
 edm_weights-improvement-01-3.2173-bigger.hdf5
 edm_weights-improvement-01-3.7982-bigger.hdf5
 edm_weights-improvement-01-4.3209-bigger.hdf5
 edm_weights-improvement-01-4.3511-bigger.hdf5
 edm_weights-improvement-01-4.4352-bigger.hdf5
 edm_weights-improvement-01-5.7362-bigger.hdf5
 edm_weights-improvement-01-5.7821-bigger.hdf5
 edm_weights-improvement-02-2.7437-bigger.hdf5
 edm_weights-improvement-02-2.8212-bigger.hdf5
 edm_weights-improvement-02-2.8536-bigger.hdf5
 edm_weights-improvement-02-2.8691-bigger.hdf5
 edm_weights-improvement-02-2.8819-bigger.hdf5
 edm_weights-improvement-02-2.9005-bigger.hdf5
 edm_weights-improvement-02-2

In [0]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [0]:
filename='jazz'
notes = []

for file in glob.glob(str(filename)+"/*.mid"):
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: 
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: 
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))




Parsing jazz/bwv772.mid
Parsing jazz/bwv801.mid


In [0]:
notes

['C4',
 'D4',
 'E4',
 'F4',
 'D4',
 'E4',
 'C4',
 'G4',
 'C3',
 'C5',
 'D3',
 'E3',
 'C5',
 'F3',
 'B4',
 'A4',
 'D3',
 'B4',
 'C5',
 'E3',
 'C3',
 'D5',
 'G3',
 'G4',
 'A4',
 'G2',
 'B4',
 'C5',
 'A4',
 'B4',
 'G4',
 'D5',
 'G3',
 'G5',
 'A3',
 'B3',
 'G5',
 'C4',
 'F5',
 'E5',
 'A3',
 'F5',
 'G5',
 'B3',
 'G3',
 'E5',
 'C4',
 'A5',
 'G5',
 'B3',
 'F5',
 'E5',
 'C4',
 'G5',
 'F5',
 'D4',
 'A5',
 'G5',
 'E4',
 'F5',
 'E5',
 'G3',
 'D5',
 'C5',
 'A3',
 'E5',
 'D5',
 'B3',
 'F5',
 'E5',
 'C4',
 'D5',
 'C5',
 'E3',
 'B4',
 'A4',
 'F#3',
 'C5',
 'B4',
 'G3',
 'D5',
 'C5',
 'A3',
 'B4',
 'A4',
 'B3',
 'G4',
 'F#4',
 'C4',
 'B3',
 'A4',
 'C4',
 'G4',
 'B4',
 'A4',
 'D3',
 'D4',
 'E3',
 'F#3',
 'C5',
 'G3',
 'B4',
 'C5',
 'E3',
 'F#3',
 '2',
 'B4',
 'G3',
 'A4',
 'G4',
 'B2',
 'F#4',
 'E4',
 'C3',
 'G4',
 'F#4',
 'D3',
 'A4',
 'G4',
 'E3',
 'B4',
 'A4',
 'F#3',
 'C5',
 'B4',
 'G3',
 'D5',
 'C5',
 'E3',
 'E5',
 'D5',
 'B2',
 'B4',
 'C5',
 'D5',
 '7.0',
 'C5',
 'D3',
 'B4',
 'C5',
 'B4',
 'A4',

In [0]:


sequence_length = 100

pitchnames = sorted(set(item for item in notes))
print(len(pitchnames))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []


for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab=len(set(notes))

network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)




80


In [0]:
#ENCODER
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')



In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 256)          525312    
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
__________

In [0]:
from keras.callbacks import EarlyStopping


filepath = str(filename)+"-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='loss',verbose=0,save_best_only=True,mode='min')
#early_stopping=EarlyStopping(patience=2)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=64, callbacks=callbacks_list)

Epoch 1/64
888/888 [==============================] - 13s 14ms/step - loss: 4.0943
Epoch 2/64
888/888 [==============================] - 11s 13ms/step - loss: 3.9093
Epoch 3/64
888/888 [==============================] - 11s 13ms/step - loss: 3.8710
Epoch 4/64
888/888 [==============================] - 11s 13ms/step - loss: 3.8609
Epoch 5/64
888/888 [==============================] - 11s 13ms/step - loss: 3.8448
Epoch 6/64
888/888 [==============================] - 13s 14ms/step - loss: 3.8035
Epoch 7/64
888/888 [==============================] - 11s 13ms/step - loss: 3.8467
Epoch 8/64
888/888 [==============================] - 11s 13ms/step - loss: 3.8087
Epoch 9/64
888/888 [==============================] - 11s 13ms/step - loss: 3.7830
Epoch 10/64
888/888 [==============================] - 11s 13ms/step - loss: 3.7706
Epoch 11/64
888/888 [==============================] - 11s 13ms/step - loss: 3.7334
Epoch 12/64
888/888 [==============================] - 11s 13ms/step - loss: 3.7194
E

KeyboardInterrupt: ignored

In [0]:
%ls

 alan_walker/
 alb/
 beethoven/
 beethoven-improvement-01-4.5143-bigger.hdf5
 beethoven-improvement-02-4.2629-bigger.hdf5
 beethoven-improvement-03-4.1827-bigger.hdf5
 beethoven-improvement-04-4.1255-bigger.hdf5
 beethoven-improvement-05-4.0874-bigger.hdf5
 beethoven-improvement-06-4.0072-bigger.hdf5
 beethoven-improvement-07-3.9590-bigger.hdf5
 beethoven-improvement-08-3.8780-bigger.hdf5
 beethoven-improvement-09-3.7331-bigger.hdf5
 beethoven-improvement-10-3.5969-bigger.hdf5
 beethoven-improvement-11-3.4838-bigger.hdf5
 beethoven-improvement-12-3.3364-bigger.hdf5
 beethoven-improvement-13-3.1758-bigger.hdf5
 beethoven-improvement-14-3.0389-bigger.hdf5
 beethoven-improvement-15-2.8837-bigger.hdf5
 beethoven-improvement-16-2.7315-bigger.hdf5
 beethoven-improvement-17-2.5745-bigger.hdf5
 beethoven-improvement-18-2.3978-bigger.hdf5
 beethoven-improvement-19-2.2540-bigger.hdf5
 beethoven-improvement-20-2.0665-bigger.hdf5
 beethoven-improvement-21-1.9102-bigger.hdf5
 beethoven-improvement-

In [0]:
#DECODER
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.load_weights('bryan_adams-improvement-50-0.1329-bigger.hdf5')



In [0]:
len(notes)

671

In [0]:

start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(notes))
print(len(int_to_note))
pattern = network_input[start]
print(pattern.shape)
prediction_output = []


for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)

    result = int_to_note[index]
    prediction_output.append(result)

    pattern=numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]



671
(100, 1)


In [0]:
prediction_output

['B-3',
 'B-3',
 'B-3',
 'C5',
 'B-3',
 'B-3',
 'B-3',
 'D5',
 'F4',
 'D5',
 'D4',
 'D5',
 'C3',
 'B-2',
 'F3',
 'C5',
 'B-4',
 'F4',
 'B-3',
 'F3',
 'E-5',
 'B-3',
 'B-4',
 'F4',
 'C3',
 'B-2',
 'F3',
 'C5',
 'B-4',
 'F4',
 'B-3',
 'F3',
 'B-4',
 'A4',
 'B-3',
 'B-4',
 'B-4',
 'C4',
 'B-3',
 'C4',
 'B-4',
 'D5',
 'D5',
 'F4',
 'F3',
 'F4',
 'F4',
 'B-3',
 'G4',
 'C4',
 'B-3',
 'B-3',
 'F3',
 'F4',
 'B-3',
 'B-3',
 'F3',
 'F3',
 'F5',
 'F5',
 'F3',
 'B-3',
 'F3',
 'F4',
 'B-3',
 'B-3',
 'F5',
 'B-4',
 'B-3',
 'G3',
 'C4',
 'C4',
 'C5',
 'B-3',
 'B-3',
 'D5',
 'D5',
 'F4',
 'B-3',
 'F3',
 'B-3',
 'G4',
 'F4',
 'D4',
 'F3',
 'B-3',
 'B-3',
 'B-3',
 'F3',
 'F4',
 'F3',
 'B-3',
 'C3',
 'B-2',
 'F5',
 'F3',
 'B-3',
 'C3',
 'B-2',
 'F3',
 'F3',
 'F4',
 'F4',
 'F3',
 'B-3',
 'D4',
 'F5',
 'B-4',
 'B-3',
 'B-4',
 'C3',
 'B-2',
 'F3',
 'C5',
 'B-3',
 'D5',
 'C3',
 'B-2',
 'F3',
 'C5',
 'B-3',
 'D5',
 'C5',
 'B-3',
 'B-3',
 'F5',
 'C4',
 'B-4',
 'F4',
 'F4',
 'C5',
 'B-3',
 'B-3',
 'B-3',
 'F5',

In [0]:
from music21 import stream

offset = 0
output_notes = []


for pattern in prediction_output:

    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)


    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='output_bryan_adams.mid')

'output_bryan_adams.mid'

In [0]:
import IPython
IPython.display.Audio('output_coldplay.mid')

# New Section

In [0]:
%ls

 alan_walker/
 alb/
 beethoven/
 beethoven-improvement-01-4.5143-bigger.hdf5
 beethoven-improvement-02-4.2629-bigger.hdf5
 beethoven-improvement-03-4.1827-bigger.hdf5
 beethoven-improvement-04-4.1255-bigger.hdf5
 beethoven-improvement-05-4.0874-bigger.hdf5
 beethoven-improvement-06-4.0072-bigger.hdf5
 beethoven-improvement-07-3.9590-bigger.hdf5
 beethoven-improvement-08-3.8780-bigger.hdf5
 beethoven-improvement-09-3.7331-bigger.hdf5
 beethoven-improvement-10-3.5969-bigger.hdf5
 beethoven-improvement-11-3.4838-bigger.hdf5
 beethoven-improvement-12-3.3364-bigger.hdf5
 beethoven-improvement-13-3.1758-bigger.hdf5
 beethoven-improvement-14-3.0389-bigger.hdf5
 beethoven-improvement-15-2.8837-bigger.hdf5
 beethoven-improvement-16-2.7315-bigger.hdf5
 beethoven-improvement-17-2.5745-bigger.hdf5
 beethoven-improvement-18-2.3978-bigger.hdf5
 beethoven-improvement-19-2.2540-bigger.hdf5
 beethoven-improvement-20-2.0665-bigger.hdf5
 beethoven-improvement-21-1.9102-bigger.hdf5
 beethoven-improvement-

In [0]:
!pip install pydub

In [0]:
random.seed(9001)
d = random.randint(1, 10)
for i in range(random.randint(1, 10)):
    if random.randint(1, 10) < 7:
        print(d)
    else:
        random.seed(9001)
        d = random.randint(1, 10)
        print(random.randint(1, 10))

1
1
5
1
1


In [0]:
directionList = [(0.0, 1.0), (1.0, 0.0), (-1.0, 0.0), (0.0, -1.0)]
myDirection = random.choice(directionList)

In [0]:
myDirection

(-1.0, 0.0)